<a href="https://colab.research.google.com/github/Nikolay1982Nikolaev/Big-Data-Bicocca/blob/main/Create_Word_Embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
FILEPATH = "/content/drive/MyDrive/00.Last_sprint_UNIVERSITA/06.BigData/"

## Import and Pre-process your corpus:

In [10]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [7]:
import pandas as pd
df = pd.read_csv(FILEPATH + 'data/tripadvisor_hotel_reviews.csv')
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [8]:
len(df)

20491

In [9]:
raw_text = df['Review']

In [11]:
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

text = raw_text.str.lower().str.replace('[^\w\s\d]',' ', regex=True) # \d tiene anche i caratteri numerici
text = text.str.split()
%time text = text.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x])
for row in text:
    [row.remove(i) for i in row if len(i) < 2]

CPU times: user 9.78 s, sys: 84.6 ms, total: 9.86 s
Wall time: 9.89 s


In [12]:
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
connectors = stopwords.words('english')


bigram = Phrases(text, min_count=5, threshold=0.2, connector_words=connectors) #gensim 4
# %time bigram = Phrases(text, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3
%time bigrams = [bigram[item] for item in text]

ngram = Phrases(bigrams, min_count=5, threshold=0.2, connector_words=connectors) # gensim 4
# %time ngram = Phrases(bigrams, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
%time ngrams = [ngram[item] for item in bigrams]

CPU times: user 4.21 s, sys: 60 ms, total: 4.27 s
Wall time: 4.58 s
CPU times: user 4.8 s, sys: 25 ms, total: 4.82 s
Wall time: 5.1 s


In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])
train_sentences = []
for row in ngrams:
    train_sentences.append([item for item in row if item not in stop])
print(train_sentences[0])

['nice_hotel', 'expensive_parking', 'got_good', 'deal_stay', 'hotel', 'anniversary', 'arrived_late', 'evening', 'took_advice', 'previous_review', 'valet_parking', 'check_quick', 'easy', 'little_disappointed', 'non_existent', 'view_room', 'room_clean', 'nice_size', 'bed_comfortable', 'woke', 'stiff', 'neck', 'high', 'pillow', 'soundproof', 'like', 'heard', 'music', 'room_night', 'morning', 'loud', 'bang', 'door_opening', 'closing', 'hear_people_talking', 'hallway', 'maybe', 'noisy', 'neighbor', 'aveda_bath', 'product_nice', 'goldfish', 'stay_nice', 'touch', 'taken_advantage', 'staying_longer', 'location_great_walking_distance', 'shopping', 'overall_nice', 'experience', 'pay_40', 'parking', 'night']


In [15]:
import csv

with open(FILEPATH +'data/tripadvisor_corups_clean.csv', 'w') as file:
  writer = csv.writer(file)
  writer.writerows(train_sentences)

## Import W2V and FastText and train a vector model

In [17]:
import csv
train_sentences = []
with open(FILEPATH + 'data/tripadvisor_corups_clean.csv', 'r') as file:
  csv_reader = csv.reader(file)
  next(csv_reader) #skip the header row
  for line in csv.reader(file):
    train_sentences.append(line)

In [19]:
from collections import Counter
c = Counter()
for sentence in train_sentences:
  c.update(sentence)

print(f'Most common words in the corpus: {c.most_common(10)}\n')
print(f'Number of different words in the corpus: {len(c)}')

Most common words in the corpus: [('hotel', 6512), ('room', 5539), ('stay', 3233), ('day', 3219), ('like', 3074), ('night', 3059), ('time', 2992), ('staff', 2528), ('place', 2523), ('resort', 2512)]

Number of different words in the corpus: 85483


In [20]:
from gensim.models import Word2Vec

#sg: training algorithm. 1 for skipgram, 0 for cbow
#hs: optimisation method. 1 for hierarchical softmax, 0 for negative sampling

# %time w2v_model = Word2Vec(sg=0, hs=1, sentences=train_sentences, size=100, window=5, min_count=5, iter=2) #gensim 3
%time w2v_model = Word2Vec(sg=0, hs=1,
                           sentences=train_sentences,
                           vector_size=100,
                           window=5,
                           min_count=10,
                           epochs=2) #gensim 4

CPU times: user 24.2 s, sys: 127 ms, total: 24.4 s
Wall time: 18.4 s


In [21]:
from gensim.models import FastText

# %time ft_model = FastText(sg=0, hs=1, sentences=train_sentences, size=100, window=5, min_count=5, iter=2, min_n=5, max_n=6) #gensim 3
%time ft_model = FastText(sg=0,
                          hs=1,
                          sentences=train_sentences,
                          vector_size=100,
                          window=5,
                          min_count=10,
                          epochs=2,
                          min_n=3,
                          max_n=6) #gensim 4

CPU times: user 1min 56s, sys: 687 ms, total: 1min 57s
Wall time: 1min 27s


In [24]:
len(w2v_model.wv) #gensim 4
# len(w2v_model.wv.vocab) #gensim 3

22785

## Save the Model to FIle

In [27]:
ft_model.save(FILEPATH  + 'models/ft_model.vec')
w2v_model.save(FILEPATH + 'models/w2v_model.vec')

In [29]:
#readable alternative (no subword information)
ft_model.wv.save_word2vec_format(FILEPATH + 'models/ft_readable.vec', binary=False)

In [31]:
#save small version
w2v_model.wv.save_word2vec_format(FILEPATH+'models/w2v_model.bin', binary=True)

## Embedding Space Navigation

In [33]:
#from gensim.test.utils import get_tmpfile
#fname = get_tmpfile("fasttext.model")

w2v_model = Word2Vec.load(FILEPATH + 'models/w2v_model.vec')
ft_model = FastText.load(FILEPATH  + 'models/ft_model.vec')

In [34]:
# print vector
ft_model.wv['room'][:10]

array([ 0.9082972 , -0.23496175,  1.2497648 , -0.37476614, -1.2075347 ,
        0.34582984, -2.7972605 , -1.7164397 , -0.06300891, -1.0634736 ],
      dtype=float32)

In [35]:
# most similar
w2v_model.wv.most_similar('italy')

[('celebrate', 0.9921784996986389),
 ('catalonia', 0.9913711547851562),
 ('pleasure_staying', 0.9912887215614319),
 ('omni', 0.9905984997749329),
 ('short_break', 0.9901502728462219),
 ('visit_amsterdam', 0.9898723363876343),
 ('better_hotel', 0.9898374676704407),
 ('bit_apprehensive', 0.9880114197731018),
 ('getting_married', 0.9879375696182251),
 ('priced_hotel', 0.9876154661178589)]

In [36]:
ft_model.wv.most_similar('breakfast')

[('breakfast_ok', 0.9960452318191528),
 ('breakfast_breakfast', 0.9953687191009521),
 ('breakfast_item', 0.9950722455978394),
 ('skip_breakfast', 0.9942587018013),
 ('hot_breakfast', 0.9934364557266235),
 ('serf_breakfast', 0.9928027391433716),
 ('free_breakfast', 0.9921733140945435),
 ('breakfast_menu', 0.992038369178772),
 ('offer_breakfast', 0.9915217757225037),
 ('fine_breakfast', 0.9914931058883667)]

In [37]:
# similarity between words

word1 = 'guitar'
word2 = 'piano'

ft_similarity = ft_model.wv.similarity(word1, word2)
w2v_similarity = w2v_model.wv.similarity(word1, word2)

print(f'fasttext similarity between {word1} and {word2}: {ft_similarity}')
print(f'Word2Vec similarity between {word1} and {word2}: {w2v_similarity}')

fasttext similarity between guitar and piano: 0.7787719964981079
Word2Vec similarity between guitar and piano: 0.9295568466186523


In [38]:
ft_model.wv.most_similar("universita", topn = 3)

[('universitat', 0.9810056686401367),
 ('ritz_carltons', 0.9793961644172668),
 ('st_regis', 0.9760870933532715)]

In [40]:
# w2v_model.most_similar("universita", topn = 3)

In [41]:
# embeddings algebra
ft_model.wv.most_similar(positive=['woman','king'], negative=['man'], topn = 10) #queen

[('asleep', 0.9637253284454346),
 ('flowing', 0.9620884656906128),
 ('sleeping', 0.959139347076416),
 ('smelling', 0.9583942890167236),
 ('singing', 0.9551006555557251),
 ('replacing', 0.9533433318138123),
 ('soaking', 0.9514635801315308),
 ('sliding', 0.9513953328132629),
 ('ironing', 0.9501166343688965),
 ('carpeting', 0.9492058753967285)]

In [42]:
ft_model.wv.similarity('queen', 'king') #il contesto è importante!!

0.7339852

In [43]:
ft_model.wv.similarity('queen', 'princess') #il contesto è importante!!

0.11790038

In [44]:
# find less correlated word in a list
ft_model.wv.doesnt_match('breakfast cereal dinner lunch'.split())
# ft_model.wv.doesnt_match('breakfast cereal dinner milk'.split())

'breakfast'